This is a notebook to generate mel-spectrograms from a TTS model to be used for WaveRNN training.

In [6]:
TTS_PATH = "/home/ricktjwong/"
!which python

/opt/conda/envs/fastdub/bin/python


In [14]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(TTS_PATH)
import torch
import importlib
import numpy as np
from tqdm import tqdm as tqdm
from torch.utils.data import DataLoader
from TTS.models.tacotron2 import Tacotron2
from TTS.datasets.TTSDataset import MyDataset
from TTS.utils.audio import AudioProcessor
from TTS.utils.visual import plot_spectrogram
from TTS.utils.generic_utils import setup_model
from TTS.utils.io import (save_best_model, save_checkpoint,
                          load_config, copy_config_file)
from TTS.datasets.preprocess import ljspeech
%matplotlib inline

import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
use_cuda = torch.cuda.is_available()
print(use_cuda)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
False


In [15]:
def set_filename(wav_path, out_path):
    wav_file = os.path.basename(wav_path)
    file_name = wav_file.split('.')[0]
    os.makedirs(os.path.join(out_path, "quant"), exist_ok=True)
    os.makedirs(os.path.join(out_path, "mel"), exist_ok=True)
    os.makedirs(os.path.join(out_path, "wav_gl"), exist_ok=True)
    wavq_path = os.path.join(out_path, "quant", file_name)
    mel_path = os.path.join(out_path, "mel", file_name)
    wav_path = os.path.join(out_path, "wav_gl", file_name)
    return file_name, wavq_path, mel_path, wav_path

In [16]:
OUT_PATH = "/home/ricktjwong/melspec"
DATA_PATH = "/home/ricktjwong/Clean16khz/"
DATASET = "tweb"
METADATA_FILE = "metadata.txt"
MODEL_FILE = "/home/ricktjwong/model_outputs/ljspeech-May-23-2020_03+51PM-0000000/best_model.pth.tar"
CONFIG_PATH = "/home/ricktjwong/TTS/config.json"
VOCODER_CONFIG_PATH = "/home/ricktjwong/neural_vocoder/WaveRNN_config_16K.json"

DRY_RUN = True   # if False, does not generate output files, only computes loss and visuals.
BATCH_SIZE = 32

use_cuda = torch.cuda.is_available()
print(" > CUDA enabled: ", use_cuda)

C = load_config(CONFIG_PATH)
C_vocoder = load_config(VOCODER_CONFIG_PATH)
ap = AudioProcessor(**C.audio)

# enable/disable some TTS options manually
C.forward_attn_mask = False

 > CUDA enabled:  False
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > num_freq:513
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:200
 | > win_length:1024
 | > n_fft:1024


In [17]:
preprocessor = importlib.import_module('TTS.datasets.preprocess')
preprocessor = getattr(preprocessor, DATASET.lower())

dataset = MyDataset(outputs_per_step=2, text_cleaner=C.text_cleaner, compute_linear_spec=False, ap=ap, meta_data=METADATA_FILE, use_phonemes=C.use_phonemes,  phoneme_cache_path=C.phoneme_cache_path)
# dataset = MyDataset(DATA_PATH, METADATA_FILE, C.r, C.text_cleaner, ap, preprocessor, use_phonemes=C.use_phonemes,  phoneme_cache_path=C.phoneme_cache_path)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=4, collate_fn=dataset.collate_fn, shuffle=False, drop_last=False)

In [22]:
from TTS.utils.text.symbols import symbols, phonemes
from TTS.utils.generic_utils import sequence_mask
from TTS.layers.losses import L1LossMasked
from TTS.utils.text.symbols import symbols, phonemes

# load the model
num_chars = len(phonemes) if C.use_phonemes else len(symbols)
model = setup_model(num_chars, num_speakers=1, c=C)
checkpoint = torch.load(MODEL_FILE, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model'])
print(checkpoint['step'])
model.eval()
if use_cuda:
    model = model.cuda()

 > Using model: Tacotron2
142258


In [23]:
!which python
import torch
torch.cuda.is_available()

/opt/conda/envs/fastdub/bin/python


False

### Generate model outputs 

In [24]:
import pickle

file_idxs = []
losses = []
postnet_losses = []
criterion = L1LossMasked(seq_len_norm=False)
for data in tqdm(loader):
    # setup input data
    text_input = data[0]
    text_lengths = data[1]
    linear_input = data[2]
    mel_input = data[3]
    mel_lengths = data[4]
    stop_targets = data[5]
    item_idx = data[6]
    
    # dispatch data to GPU
    if use_cuda:
        text_input = text_input.cuda()
        text_lengths = text_lengths.cuda()
        mel_input = mel_input.cuda()
        mel_lengths = mel_lengths.cuda()
#         linear_input = linear_input.cuda()
        stop_targets = stop_targets.cuda()
    
    mask = sequence_mask(text_lengths)
    mel_outputs, postnet_outputs, alignments, stop_tokens = model.forward(text_input, text_lengths, mel_input)
    
    # compute mel specs from linear spec if model is Tacotron
    mel_specs = []
    if C.model == "Tacotron":
        postnet_outputs = postnet_outputs.data.cpu().numpy()
        for b in range(postnet_outputs.shape[0]):
            postnet_output = postnet_outputs[b]
            mel_specs.append(torch.FloatTensor(ap.out_linear_to_mel(postnet_output.T).T).cuda())
    postnet_outputs = torch.stack(mel_specs)
    
    loss = criterion(mel_outputs, mel_input, mel_lengths)
    loss_postnet = criterion(postnet_outputs, mel_input, mel_lengths)
    losses.append(loss.item())
    postnet_losses.append(loss_postnet.item())
    if not DRY_RUN:
        for idx in range(text_input.shape[0]):
            wav_file_path = item_idx[idx]
            wav = ap.load_wav(wav_file_path)
            file_name, wavq_path, mel_path, wav_path = set_filename(wav_file_path, OUT_PATH)
            file_idxs.append(file_name)

            # quantize and save wav
            if type(C_vocoder.mode) is int and C_vocoder.mulaw:
                wavq = ap.mulaw_encode(wav, C_vocoder.mode)
                np.save(wavq_path, wavq, allow_pickle=False)
            elif type(C_vocoder.mode) is int:
                wavq = ap.quantize(wav)
                np.save(wavq_path, wavq, allow_pickle=False)

            # save TTS mel
            mel = postnet_outputs[idx]
            mel = mel.data.cpu().numpy()
            mel_length = mel_lengths[idx]
            mel = mel[:mel_length, :].T
            np.save(mel_path, mel)

            # save GL voice
    #         wav_gen = ap.inv_mel_spectrogram(mel.T) # mel to wav
    #         wav_gen = ap.quantize(wav_gen)
    #         np.save(wav_path, wav_gen)

if not DRY_RUN:
    pickle.dump(file_idxs, open(OUT_PATH+"/dataset_ids.pkl", "wb"))      
    

print(np.mean(losses))
print(np.mean(postnet_losses))

  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/fastdub/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/envs/fastdub/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/fastdub/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/ricktjwong/TTS/datasets/TTSDataset.py", line 169, in __getitem__
    return self.load_data(idx)
  File "/home/ricktjwong/TTS/datasets/TTSDataset.py", line 114, in load_data
    text, wav_file, speaker_name = self.items[idx]
ValueError: not enough values to unpack (expected 3, got 1)


### Check model performance

In [ ]:
idx = 1
mel_example = postnet_outputs[idx].data.cpu().numpy()
plot_spectrogram(mel_example[:mel_lengths[idx], :], ap);
print(mel_example[:mel_lengths[1], :].shape)

In [ ]:
mel_example = mel_outputs[idx].data.cpu().numpy()
plot_spectrogram(mel_example[:mel_lengths[idx], :], ap);
print(mel_example[:mel_lengths[1], :].shape)

In [ ]:
wav = ap.load_wav(item_idx[idx])
melt = ap.melspectrogram(wav)
print(melt.shape)
plot_spectrogram(melt.T, ap);

In [ ]:
# postnet, decoder diff
from matplotlib import pylab as plt
mel_diff = mel_outputs[idx] - postnet_outputs[idx]
plt.figure(figsize=(16, 10))
plt.imshow(abs(mel_diff.detach().cpu().numpy()[:mel_lengths[idx],:]).T,aspect="auto", origin="lower");
plt.colorbar()
plt.tight_layout()

In [ ]:
from matplotlib import pylab as plt
# mel = mel_poutputs[idx].detach().cpu().numpy()
mel = postnet_outputs[idx].detach().cpu().numpy()
mel_diff2 = melt.T - mel[:melt.shape[1]]
plt.figure(figsize=(16, 10))
plt.imshow(abs(mel_diff2).T,aspect="auto", origin="lower");
plt.colorbar()
plt.tight_layout()